# **FIRST BASELINE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install datasets

In [ ]:
!pip install rouge_score

In [ ]:
import numpy as np 
import pandas as pd 
from datasets import load_metric
import re

In [ ]:
df_test = pd.read_csv('/content/drive/My Drive/DL 2/test.csv')
df_test = df_test.drop(['id'],axis=1)

In [ ]:
df_test.head()

,article,highlights
0,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."


In [ ]:
metric = load_metric("rouge")

In [ ]:
def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

In [ ]:
ref_summaries = list(df_test['highlights'])

In [ ]:
for i in range (3):
    candidate_summaries = list(df_test['article'].apply(lambda x: ' '.join(re.split(r'(?<=[.:;])\s', x)[:i+1])))
    print(f"First {i+1} senctences: Scores {calc_rouge_scores(candidate_summaries, ref_summaries)}")

First 1 senctences: Scores {'rouge1': 26.9, 'rouge2': 9.4, 'rougeL': 18.4, 'rougeLsum': 23.2}
First 2 senctences: Scores {'rouge1': 37.6, 'rouge2': 15.4, 'rougeL': 23.4, 'rougeLsum': 31.1}
First 3 senctences: Scores {'rouge1': 39.9, 'rouge2': 17.3, 'rougeL': 24.9, 'rougeLsum': 32.8}


# **ZERO SHOT MODEL**

In [ ]:
! pip install transformers

In [ ]:
from transformers import pipeline
from google.colab import files

In [ ]:
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
cutoff_summary = 5
cutoff_text = 256
df_test= df_test[(df_test['highlights'].apply(lambda x: len(x.split()) >= cutoff_summary)) & (df_test['article'].apply(lambda x: len(x.split()) <= cutoff_text))]

In [ ]:
ref_summaries = list(df_test['highlights'])
len(ref_summaries)

In [ ]:
# import pandas as pd
# df_test = pd.read_csv('data/test.csv')
# ref_summaries = list(df_test['summary'])
texts = list(df_test['article'])


In [ ]:
texts[5]

"Three police officers have been sent forward for trial after being accused of stealing bundles of notes worth £30,000 from the home of a suspected criminal. Detective Sergeant Stephen Phillips, 46, and police constables Christopher Evans, 37, and Michael Stokes, 34 are all facing charges of 'theft by a serving police officer'. Cardiff Crown Court heard that the three officers from South Wales Police were all arrested as a result of an investigation being carried out by the force's professional standards department. Philips, of Winch Wen, Swansea, Evans of Llangennech, Carmarthenshire and Stokes from Glynneath were released on bail following today's hearing to return to Cardiff Crown Court to stand trial on June 15. Cardiff Crown Court heard that Stephen Phillips, left, Christopher Evans, centre, and Michael Stokes, right, are all accused of taking bundles of cash worth £30,000 from the home of a suspected criminal . Sorry we are not currently accepting comments on this article."

In [ ]:
summarizer(texts[5], max_length=40)

[{'summary_text': "detective sgt Stephen Phillips, 46, and police constables Christopher Evans, 37, and Michael Stokes, 34 are all facing charges of 'theft by a"}]

In [ ]:
candidate_summaries = []

for text in texts:
  print(type(text))
  candidate = summarizer(text, min_length=5, max_length=40)
  candidate_summaries.append(candidate[0]['summary_text'])

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

In [ ]:
file = open("zero-shot-summaries.txt", "w")
for s in candidate_summaries:
    file.write(s + "\n")
file.close()

In [ ]:
candidate_summaries[:5]

['Dougie Freedman is on the verge of agreeing a new two-year deal . he replaced cult hero Stuart Pearce at the city ground',
 "alastair cook completed a much-needed century on the second morning of England's opening tour match in the west indies . cook resumed on 95 and reached three figures with",
 'deShane Beckford has impressed on trial with westham . the 17-year-old has also been linked with a host of european clubs . west ham have revealed',
 'Tim Swiel will return to Harlequins on a full-time contract . the 21-year-old spent four months on loan with the sharks earlier this season',
 '"Avengers" character is a master with the bow and arrow . "Listen I\'ve got powers too, and they\'re pretty sweet," he ']

In [ ]:
len(candidate_summaries)

In [ ]:
# from datasets import load_metric
metric = load_metric("rouge")

In [ ]:
files.download("zero-shot-summaries.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
calc_rouge_scores(candidate_summaries, ref_summaries)

{'rouge1': 39.7, 'rouge2': 20.7, 'rougeL': 30.2, 'rougeLsum': 34.9}

# **HYPER-TUNING A SUMMARISATION MODEL**

In [ ]:
! pip install simplet5 -q

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import torch
import re
import os
import string
import pandas as pd
from simplet5 import SimpleT5
from google.colab import files
from datasets import load_metric

In [ ]:
class Settings:

    MODEL_TYPE = "t5"
    MODEL_NAME = "t5-base"

    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    TRAIN_DATA = "/content/drive/My Drive/DL 2/train.csv"
    Columns = ['highlights', 'article']

    USE_GPU = None
    if str(DEVICE) == "cuda":
        USE_GPU=True
    else:
        USE_GPU = False

    EPOCHS = 10  # can be increased to 30

    encoding = 'latin-1'
    columns_dict = {"highlights": "target_text", "article": "source_text"}
    df_column_list = ['source_text', 'target_text']
    SUMMARIZE_KEY = "summarize: "
    SOURCE_TEXT_KEY = 'source_text'
    TEST_SIZE = 0.2
    BATCH_SIZE = 8       #check at 32 also
    source_max_token_len = 256
    target_max_token_len = 50
    train_df_len = 5000
    test_df_len = 100

In [ ]:
class Preprocess:
    def __init__(self):
        self.settings = Settings

    def clean_text(self, text):
        text = text.lower()
        text = re.sub('\[.*?\]', '', text)
        text = re.sub('https?://\S+|www\.\S+', '', text)
        text = re.sub('<.*?>+', '', text)
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
        text = re.sub('\n', '', text)
        text = re.sub('\w*\d\w*', '', text)
        return text

    def preprocess_data(self, data_path):
        df = pd.read_csv(data_path, encoding=self.settings.encoding, usecols=self.settings.Columns)
        # simpleT5 expects dataframe to have 2 columns: "source_text" and "target_text"
        df = df.rename(columns=self.settings.columns_dict)
        df = df[self.settings.df_column_list]
        # T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
        df[self.settings.SOURCE_TEXT_KEY] = self.settings.SUMMARIZE_KEY + df[self.settings.SOURCE_TEXT_KEY]

        return df

In [ ]:
class T5Model:
    def __init__(self, model_type, model_name):
        self.model = SimpleT5()
        self.model.from_pretrained(model_type=model_type,
                                   model_name=model_name)

    def load_model(self, model_type, model_path, use_gpu: bool):
        try:
            self.model.load_model(
                model_type=model_type,
                model_dir=model_path,
                use_gpu=use_gpu
            )

        except BaseException as ex:
            print("error occurred while loading model ", str(ex))
            

In [ ]:
class Train:
    def __init__(self):
        # initialize required class
        self.settings = Settings
        self.preprocess = Preprocess()

        # initialize required variables
        self.t5_model = None

    def __initialize(self):
        try:
            self.t5_model = T5Model(model_name=self.settings.MODEL_NAME,
                                    model_type=self.settings.MODEL_TYPE)

        except BaseException as ex:
            print("error occurred while loading model ", str(ex))

    def set_seed(self, seed_value = 42):
        random.seed(seed_value) # can change seed value
        np.random.seed(seed_value)
        torch.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
    
    def train(self, df):
        try:
            train_df, test_df = train_test_split(df, test_size=self.settings.TEST_SIZE)

            self.t5_model.model.train(train_df=train_df[:self.settings.train_df_len],
                                      eval_df=test_df[:self.settings.test_df_len],
                                      source_max_token_len=self.settings.source_max_token_len,
                                      target_max_token_len=self.settings.target_max_token_len,
                                      batch_size=self.settings.BATCH_SIZE, max_epochs=self.settings.EPOCHS,
                                      use_gpu=self.settings.USE_GPU)
        except BaseException as ex:
          print("error occurred while loading model ", str(ex))

    def run(self):
        try:
            print("Loading and Preparing the Dataset-----!! ")
            df = self.preprocess.preprocess_data(self.settings.TRAIN_DATA)
            print(df.head())
            print("Dataset Successfully Loaded and Prepared-----!! ")
            print("Loading and Initializing the T5 Model -----!! ")
            self.__initialize()
            print("Model Successfully Loaded and Initialized-----!! ")
            print("------------------Starting Training-----------!!")
            self.set_seed()
            self.train(df)
            print("Training complete-----!!!")

        except BaseException as ex:
            print("Following Exception Occurred---!! ", str(ex))

In [ ]:
t= Train()
t.run()

Loading and Preparing the Dataset-----!! 
                                         source_text  \
0  summarize: By . Associated Press . PUBLISHED: ...   
1  summarize: (CNN) -- Ralph Mata was an internal...   
2  summarize: A drunk driver who killed a young w...   
3  summarize: (CNN) -- With a breezy sweep of his...   
4  summarize: Fleetwood are the only team still t...   

                                         target_text  
0  Bishop John Folda, of North Dakota, is taking ...  
1  Criminal complaint: Cop used his role to help ...  
2  Craig Eccleston-Todd, 27, had drunk at least t...  
3  Nina dos Santos says Europe must be ready to a...  
4  Fleetwood top of League One after 2-0 win at S...  
Dataset Successfully Loaded and Prepared-----!! 
Loading and Initializing the T5 Model -----!! 


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Training complete-----!!!


In [ ]:
! ( cd outputs; ls )

simplet5-epoch-0-train-loss-1.9043-val-loss-1.7073
simplet5-epoch-1-train-loss-1.6624-val-loss-1.7131
simplet5-epoch-2-train-loss-1.4894-val-loss-1.7391
simplet5-epoch-3-train-loss-1.3458-val-loss-1.7753
simplet5-epoch-4-train-loss-1.2245-val-loss-1.8209
simplet5-epoch-5-train-loss-1.1137-val-loss-1.8812
simplet5-epoch-6-train-loss-1.0222-val-loss-1.9599
simplet5-epoch-7-train-loss-0.9329-val-loss-2.0052
simplet5-epoch-8-train-loss-0.8605-val-loss-2.0721
simplet5-epoch-9-train-loss-0.7905-val-loss-2.1557


In [ ]:
!zip -r /content/outputs.zip /content/outputs

  adding: content/outputs/ (stored 0%)
  adding: content/outputs/simplet5-epoch-8-train-loss-0.8605-val-loss-2.0721/ (stored 0%)
  adding: content/outputs/simplet5-epoch-8-train-loss-0.8605-val-loss-2.0721/tokenizer.json (deflated 74%)
  adding: content/outputs/simplet5-epoch-8-train-loss-0.8605-val-loss-2.0721/special_tokens_map.json (deflated 83%)
  adding: content/outputs/simplet5-epoch-8-train-loss-0.8605-val-loss-2.0721/pytorch_model.bin (deflated 7%)
  adding: content/outputs/simplet5-epoch-8-train-loss-0.8605-val-loss-2.0721/spiece.model (deflated 48%)
  adding: content/outputs/simplet5-epoch-8-train-loss-0.8605-val-loss-2.0721/tokenizer_config.json (deflated 80%)
  adding: content/outputs/simplet5-epoch-8-train-loss-0.8605-val-loss-2.0721/config.json (deflated 63%)
  adding: content/outputs/simplet5-epoch-7-train-loss-0.9329-val-loss-2.0052/ (stored 0%)
  adding: content/outputs/simplet5-epoch-7-train-loss-0.9329-val-loss-2.0052/tokenizer.json (deflated 74%)
  adding: content/o

In [ ]:
!zip -r  /content/outputs_epoch9.zip /content/outputs/simplet5-epoch-9-train-loss-0.7905-val-loss-2.1557

  adding: content/outputs/simplet5-epoch-9-train-loss-0.7905-val-loss-2.1557/ (stored 0%)
  adding: content/outputs/simplet5-epoch-9-train-loss-0.7905-val-loss-2.1557/tokenizer.json (deflated 74%)
  adding: content/outputs/simplet5-epoch-9-train-loss-0.7905-val-loss-2.1557/special_tokens_map.json (deflated 83%)
  adding: content/outputs/simplet5-epoch-9-train-loss-0.7905-val-loss-2.1557/pytorch_model.bin (deflated 7%)
  adding: content/outputs/simplet5-epoch-9-train-loss-0.7905-val-loss-2.1557/spiece.model (deflated 48%)
  adding: content/outputs/simplet5-epoch-9-train-loss-0.7905-val-loss-2.1557/tokenizer_config.json (deflated 80%)
  adding: content/outputs/simplet5-epoch-9-train-loss-0.7905-val-loss-2.1557/config.json (deflated 63%)


In [ ]:
!zip -r  /content/outputs_epoch5.zip /content/outputs/simplet5-epoch-5-train-loss-1.1137-val-loss-1.8812

  adding: content/outputs/simplet5-epoch-5-train-loss-1.1137-val-loss-1.8812/ (stored 0%)
  adding: content/outputs/simplet5-epoch-5-train-loss-1.1137-val-loss-1.8812/tokenizer.json (deflated 74%)
  adding: content/outputs/simplet5-epoch-5-train-loss-1.1137-val-loss-1.8812/special_tokens_map.json (deflated 83%)
  adding: content/outputs/simplet5-epoch-5-train-loss-1.1137-val-loss-1.8812/pytorch_model.bin (deflated 7%)
  adding: content/outputs/simplet5-epoch-5-train-loss-1.1137-val-loss-1.8812/spiece.model (deflated 48%)
  adding: content/outputs/simplet5-epoch-5-train-loss-1.1137-val-loss-1.8812/tokenizer_config.json (deflated 80%)
  adding: content/outputs/simplet5-epoch-5-train-loss-1.1137-val-loss-1.8812/config.json (deflated 63%)


In [ ]:

files.download('/content/outputs.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/outputs_epoch5.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/outputs_epoch9.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
best_weight = "./outputs/simplet5-epoch-9-train-loss-0.7905-val-loss-2.1557" #1

In [ ]:
t5_model = T5Model(model_name=Settings.MODEL_NAME,model_type=Settings.MODEL_TYPE)
t5_model.load_model(model_type=Settings.MODEL_TYPE,
                   use_gpu=Settings.USE_GPU,
                   model_path=best_weight
                   )

In [ ]:
df_test = pd.read_csv('/content/drive/My Drive/DL 2/test.csv')
df_test = df_test.drop(['id'],axis=1)

In [ ]:
cutoff_summary = 5
cutoff_text = 180
df_test= df_test[(df_test['highlights'].apply(lambda x: len(x.split()) >= cutoff_summary)) & (df_test['article'].apply(lambda x: len(x.split()) <= cutoff_text))]

In [ ]:
ref_summaries = list(df_test['highlights'])
len(ref_summaries)

In [ ]:
texts = list(df_test['article'])

In [ ]:
t5_model.model.predict(texts[5])

["Detective Sergeant Stephen Phillips, 46, and police constables Christopher Evans, 37, and Michael Stokes, 34, all face charges of 'theft by a serving police officer' Police were all arrested as a result of an investigation"]

In [ ]:
candidate_summaries = []

for text in texts:
  print(type(text))
  candidate = t5_model.model.predict(text)
  candidate_summaries.append(candidate)

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

In [ ]:
new_arr = np.array(candidate_summaries)   
file = open("hypertuned-summaries.txt", "w+") 
content = str(new_arr)
file.write(content)
file.close()

In [ ]:
candidate_summaries[:5]

[["Dougie Freedman is on the verge of agreeing a new two-year deal at Nottingham Forest. Freedman replaced Stuart Pearce in February and has impressed at the City Ground. Forest's owners are"],
 ["England captain Alastair Cook completed a much-needed century on the second morning of England's opening tour match in the West Indies. Cook resumed on 95 and reached three figures with minimal fuss before retiring out "],
 ["West Ham are closing in on the signature of Jamaican starlet DeShane Beckford. The 17-year-old was impressed on trial with West Ham's academy. West Ham have revealed season tickets at the Olympic Stadium will cost as little"],
 ['Tim Swiel will return to Harlequins on a full-time contract. Fly-half spent four months on loan with Quins earlier this term. Swiel made 10 appearances for the Sharks during his loan spell'],
 ['"Avengers" character Jeremy Renner is the star of "Thinking Out Loud" He\'s also known for his collection of scarves and berets. The actor has appeared 

In [ ]:
files.download("sample1.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
metric = load_metric("rouge")

In [ ]:
def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

In [ ]:
calc_rouge_scores(candidate_summaries, ref_summaries)

{'rouge1': 43.2, 'rouge2': 21.4, 'rougeL': 31.6, 'rougeLsum': 37.2}